In [ ]:
import warnings
warnings.filterwarnings('ignore')
from skExSTraCS import ExSTraCS
from skrebate import ReliefF
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import learning_curve 
#Load Data Using Pandas
data = pd.read_csv("data/histo_feature.csv") #REPLACE with your own dataset .csv filename
classLabel = "conclusion"
data = data.iloc[:,9:]
data = data.drop("datetime",axis=1)
data = data.drop(data[data["conclusion"]=="OTHER"].index)
data = data.drop(data[data["conclusion"]=="UNCLEAR"].index)
dataFeatures = data.iloc[:,1:].values #DEFINE classLabel variable as the Str at the top of your dataset's action column
dataPhenotypes_raw = data.iloc[:,0].values
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(dataPhenotypes_raw)
dataPhenotypes = label_encoder.transform(dataPhenotypes_raw)

In [ ]:
#Shuffle Data Before CV
formatted = np.insert(dataFeatures,dataFeatures.shape[1],dataPhenotypes,1)
np.random.shuffle(formatted)
dataFeatures = np.delete(formatted,-1,axis=1)
dataPhenotypes = formatted[:,-1]

#Get Feature Importance Scores to use as Expert Knowledge (see https://github.com/EpistasisLab/scikit-rebate/ for more details on skrebate package)
relieff = ReliefF()
relieff.fit(dataFeatures,dataPhenotypes)
scores = relieff.feature_importances_

#Initialize ExSTraCS Model
model = ExSTraCS(learning_iterations=100, N=100,expert_knowledge=scores)
#model = ExSTraCS(N=3000)
trainedModel = model.fit(dataFeatures,dataPhenotypes)
print("Training Score: ", trainedModel.score(dataFeatures,dataPhenotypes))
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
cross_val_scoring = cross_val_score(trainedModel, dataFeatures, dataPhenotypes, cv=cv, scoring="accuracy", n_jobs=8)
print("Cross-Validation Scores: ", cross_val_scoring)

train_sizes, train_scores, test_scores = learning_curve(model, dataFeatures, dataPhenotypes, cv=cv, scoring="accuracy", n_jobs=8)

In [ ]:
# Evalution de la cross-val
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
print("Cross-Validation Scores:")
cross_val_scoring = cross_val_score(model, dataFeatures, dataPhenotypes, cv=cv, scoring="accuracy", n_jobs=-1)
print(cross_val_scoring)
plt.plot(range(5), cross_val_scoring, 'o-', color="r",
                 label="Cross-Validation Accuracy")
ylim = plt.ylim(0,1)
xticks = plt.xticks(range(5))

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(model, dataFeatures, dataPhenotypes, cv=cv, scoring="accuracy")

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
plt.ylim(0.4,1.05)
plt.title("Learning Curve (SVM). Mean Accuracy +- 1 std")
plt.xlabel("Training Examples")
plt.ylabel("Accuracy Score")
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
plot_confusion_matrix(model, dataFeatures, dataPhenotypes, display_labels=["CNM","COM","NM"])